# Utils

In the submodule modules we can find a couple usefull functions fro different purposes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tvsclib.identification as ident
import tvsclib.utils as utils
import tvsclib.approximation as approximation

from tempfile import TemporaryFile

#create a rank-1 matrix
T = np.arange(0,12).reshape((-1,1))@np.arange(0,12).reshape((1,-1))
dims_in =  np.array([2, 1, 2, 1])*2
dims_out = np.array([1, 2, 1, 2])*2
system = ident.identify(T,dims_in,dims_out,epsilon=1e-14)

## Plotting system

Using `show_system` we can get graphical representations of the system.

In [ ]:
utils.show_system(system)

We can also deactivate the shading of the D-matircies

In [ ]:
utils.show_system(system,mark_D=False)

We can also give an axis object where the representation should be plotted. 
Also note how the utils displays the seperation lines for casual and anticausal systems.

In [ ]:
fig,axs = plt.subplots(1,2)
utils.show_system(system.causal_system    ,ax=axs[0])
utils.show_system(system.anticausal_system,ax=axs[1])

# Check system

The function `check_dims` can be used to check if the dimensions of the matricies match.
This is usefull for manually creating systems.

In [ ]:
utils.check_dims(system)

Now corrupt the system:

In [ ]:
system_defect = system.causal_system.copy()
system_defect.stages[3].A_matrix = np.eye(4)

utils.check_dims(system_defect)

Usually `check_dims` assumes that the input and output states are zero-dimensional. We can override this by setting the input and output dimensions by setting the arguments `dim_state_in` and `dim_state_out`

## Save and load system

It is possible to save a system into a `.npz` file and load it back.
The function does not require pickle.

In [ ]:
file = TemporaryFile()
utils.save_system(file,system)
file.seek(0)

In [ ]:
system_load = utils.load_system(file)
np.allclose(system_load.to_matrix(),system.to_matrix())

It is also possible to save the singular values that can be used for approxiations alongside in the same file

In [ ]:
system,sigmas = ident.identify(T,dims_in,dims_out,compute_sigmas=True)
utils.save_system(file,system,sigmas=sigmas)
file.seek(0);

Now we can load it and create an `Approximation` object.

In [ ]:
system_load,sigmas_load = utils.load_system(file,load_sigmas=True)
approx = approximation.Approximation(system_load,sigmas_load)

In [ ]:
system_approx = approx.get_approxiamtion(3)